In [39]:
import os
import re
import codecs

In [40]:
def zero_digits(s):
    """
    Replace all digits in a string with zeros.
    """
    return re.sub('\d', '0', s)

def load_sentences(path):
    """
    Load sentences. A line must contain at least a word and its tag.
    Sentences are separated by empty lines.
    """
    sentences = []
    sentence = []
    for line in codecs.open(path, 'r', 'utf8'):
        line = zero_digits(line.rstrip())
        if not line:
            if len(sentence) > 0:
                if 'DOCSTART' not in sentence[0][0]:
                    sentences.append(sentence)
                sentence = []
        else:
            word = line.split()
            assert len(word) >= 2
            sentence.append(word)
    if len(sentence) > 0:
        if 'DOCSTART' not in sentence[0][0]:
            sentences.append(sentence)
    return sentences

In [41]:
def create_dico(item_list):
    """
    Create a dictionary of items from a list of list of items.
    """
    assert type(item_list) is list
    dico = {}
    for items in item_list:
        for item in items:
            if item not in dico:
                dico[item] = 1
            else:
                dico[item] += 1
    return dico

In [42]:
def create_mapping(dico):
    """
    Create a mapping (item to ID / ID to item) from a dictionary.
    Items are ordered by decreasing frequency.
    """
    sorted_items = sorted(dico.items(), key=lambda x: (-x[1], x[0]))
    id_to_item = {i: v[0] for i, v in enumerate(sorted_items) if v[1] > 2}
    item_to_id = {v: k for k, v in id_to_item.items()}
    return item_to_id, id_to_item

def word_mapping(sentences):
    """
    Create a dictionary and a mapping of words, sorted by frequency.
    """
    words = [[x[0] for x in s] for s in sentences]
    dico = create_dico(words)
    dico['<UNK>'] = 10000000
    word_to_id, id_to_word = create_mapping(dico)
    print("Found %i unique words (%i in total)" % (
        len(dico), sum(len(x) for x in words))
    )
    return dico, word_to_id, id_to_word

def tag_mapping(sentences):
    """
    Create a dictionary and a mapping of tags, sorted by frequency.
    """
    tags = [[word[-1] for word in s] for s in sentences]
    dico = create_dico(tags)
    tag_to_id, id_to_tag = create_mapping(dico)
    print("Found %i unique named entity tags" % len(dico))
    return dico, tag_to_id, id_to_tag

In [43]:
def prepare_dataset(sentences, mode=None, word_to_id=None, tag_to_id=None):
    """
    Prepare the dataset. Return 'data', which is a list of lists of dictionaries containing:
        - words (strings)
        - word indexes
        - tag indexes
    """
    assert mode == 'train' or (mode == 'test' and word_to_id and tag_to_id)

    if mode=='train':
        word_dic, word_to_id, id_to_word = word_mapping(sentences)
        tag_dic, tag_to_id, id_to_tag = tag_mapping(sentences)

    def f(x): return x
    data = []
    for s in sentences:
        str_words = [w[0] for w in s]
        words = [word_to_id[f(w) if f(w) in word_to_id else '<UNK>']
                 for w in str_words]
        tags = [tag_to_id[w[-1]] for w in s]
        data.append({
            'str_words': str_words,
            'words': words,
            'tags': tags,
        })

    if mode == 'train':
        return data, {'word_to_id':word_to_id, 'id_to_word':id_to_word, 'tag_to_id':tag_to_id, 'id_to_tag':id_to_tag}
    else:
        return data

In [44]:
import os
import re
import numpy as np
from sklearn import linear_model
from sklearn.preprocessing import OneHotEncoder
from scipy import sparse
import collections
import codecs

In [45]:
class HMM(object):
    """
     HMM, Model
    """
    def __init__(self, dic, decode_type):
        """
        Initialize the model.
        """

        self.num_words = len(dic['word_to_id'])
        self.num_tags = len(dic['tag_to_id'])

        self.initial_prob = np.ones([self.num_tags])
        self.transition_prob = np.ones([self.num_tags, self.num_tags])
        self.emission_prob = np.ones([self.num_tags, self.num_words])
        self.decode_type = decode_type

        return

    def train(self, corpus):
        """
        TODO: Train a bigram HMM model using MLE estimates.
        Complete the code to compute self,initial_prob, self.transition_prob and self.emission_prob appropriately.

        Args:
            corpus is a list of dictionaries of the form:
            {'str_words': str_words,   ### List of string words
            'words': words,            ### List of word IDs
            'tags': tags}              ### List of tag IDs
            All three lists above have length equal to the sentence length for each instance.

        """

        # Step 1: Compute initial_probs.
        # initial_prob[x]: the probability of tag x to be assigned to the first token in a sentence.
        self.initial_prob = np.zeros([self.num_tags])
        print(self.initial_prob)
        for sentence in corpus:
            # TODO: update self.initial_prob
            # (5 points)
            # START HERE
            tag = sentence['tags']
            self.initial_prob[tag[0]] += 1
            # END HERE
        print(self.initial_prob)

        # Normarlize initial_prob to sum to 1
        self.initial_prob /= np.sum(self.initial_prob)

        # Step 2: Complete the code to compute transition_prob.
        # transition_prob[x][y]: the probability of tag y to appear after tag x.
        self.transition_prob = np.zeros([self.num_tags, self.num_tags])
        for sentence in corpus:
            tag = sentence['tags']
            for i in range(1, len(tag)):
                # TODO: update self.transition_prob
                # (5 points)
                # START HERE
                 self.transition_prob = np.zeros([self.num_tags, self.num_tags])
            for sentence in corpus:
                tag = sentence['tags']
                for i in range(1, len(tag)):
                    self.transition_prob[tag[i-1]][tag[i]] += 1
                # END HERE
        print(self.transition_prob)

        # Normalize every row of transition_prob to sum to 1.
        for p in self.transition_prob:
            p /= np.sum(p)


        # Step 3: Complete the code to compute emission_prob
        # emission_prob[x][y]: the probability of word y to appear given tag x.
        # Note that for each sentence s in the corpus, word IDs are in s['words'].
        self.emission_prob = np.zeros([self.num_tags, self.num_words])
        for sentence in corpus:
            for i in range(len(sentence['tags'])):
                # TODO: update self.emission_prob
                # (5 points)
                # START HERE
                tag = sentence['tags'][i]
                word = sentence['words'][i]
                self.emission_prob[tag][word] += 1
                # END HERE
        print(self.emission_prob)

        # For every tag, normalize the emission_prob to sum to 1.
        for p in self.emission_prob:
            p /= np.sum(p)

        return

    def greedy_decode(self, sentence):
        """
        Decode a single sentence in Greedy fashion
        Return a list of tags.
        """
        tags = []

        init_scores = [self.initial_prob[t] * self.emission_prob[t][sentence[0]] for t in range(self.num_tags)]
        tags.append(np.argmax(init_scores))

        for w in sentence[1:]:
            scores = [self.transition_prob[tags[-1]][t] * self.emission_prob[t][w] for t in range(self.num_tags)]
            tags.append(np.argmax(scores))

        assert len(tags) == len(sentence)
        return tags

    def viterbi_decode(self, sentence):
        """
        TODO: Complete the code to decode a single sentence using the Viterbi algorithm.
        Args:
             sentence -- a list of ints that represents word IDs.
        Output:
             tags     -- a list of ints that represents the tags decoded from the input.
        """
        tags = []
        l = len(sentence)

        pi = self.initial_prob
        A = self.transition_prob
        O = self.emission_prob

        # Let M be the state matrix.
        # M[i,j]: most probable sequence of tags ending with tag j at the i-th token.
        M = np.zeros((l, self.num_tags))
        M[:,:] = float('-inf')

        # Use B to track the path to reach the most probable sequence.
        # B[i,j] is the tag of the (i-1)-th token in the most probable sequence ending with tag j at the i-th token.
        B = np.zeros((l, self.num_tags), 'int')

        # Compute the probability to assign each tag to the first token.
        M[0, :] = pi * O[:, sentence[0]]

        # Dynamic programming.
        for i in range(1, l):
            for j in range(self.num_tags):
                # TODO: Compute M[i, j] and B[i, j].
                # (10 points)
                # START HERE
                max_prob = float('-inf')
                max_tag = None
                for k in range(self.num_tags):
                  prob = M[i-1, k] * A[k, j] * O[j, sentence[i]]
                  if prob > max_prob:
                    max_prob = prob
                    max_tag = k
                M[i, j] = max_prob
                B[i, j] = max_tag
                # END HERE

        # Extract the optimal sequence of tags from B.
        # Start from the last position, and iteratively find the tag of each position that results in the most probable tag sequence.
        tags.append(np.argmax(M[l-1,:]))
        for i in range(l-1, 0, -1):
            # TODO: Extract the tag of the (i-1)-th token that results in the most probable sequence of tags.
            # (5 points)
            # START HERE
            tags.append(B[i, tags[-1]])
            # END HERE

        return tags

    def tag(self, sentence):
        """
        Tag a sentence using a trained HMM.
        """
        if self.decode_type == 'viterbi':
            return self.viterbi_decode(sentence)
        elif self.decode_type == 'greedy':
            return self.greedy_decode(sentence)
        else:
            raise ValueError("Unknown decoding type")

In [46]:
import os
import time
import codecs
import json
from tqdm import tqdm
import numpy as np
import collections
from sklearn.metrics import f1_score, confusion_matrix

In [47]:
def tag_corpus(model, test_corpus, output_file, dic):
    if output_file:
        f_output = codecs.open(output_file, 'w', 'utf-8')
    start = time.time()

    num_correct = 0.
    num_total = 0.
    y_pred=[]
    y_actual=[]
    print('Tagging...')
    for i, sentence in enumerate(tqdm(test_corpus)):
        tags = model.tag(sentence['words'])
        str_tags = [dic['id_to_tag'][t] for t in tags]
        y_pred.extend(tags)
        y_actual.extend(sentence['tags'])

        # Check accuracy.
        num_correct += np.sum(np.array(tags) == np.array(sentence['tags']))
        num_total += len([w for w in sentence['words']])

        if output_file:
            f_output.write('%s\n' % ' '.join('%s%s%s' % (w, '__', y)
                                             for w, y in zip(sentence['str_words'], str_tags)))

    print('---- %i lines tagged in %.4fs ----' % (len(test_corpus), time.time() - start))
    if output_file:
        f_output.close()

    print("Overall accuracy: %s\n" % (num_correct/num_total))
    return y_pred,y_actual

In [48]:
def compute_score(y_pred,y_actual):
    A = confusion_matrix(y_actual, y_pred)
    f1 = f1_score(y_actual, y_pred,average=None)
    print("Confusion Matrix:\n", A)
    print("F-1 scores: ", f1)

In [49]:
def runHiddenMarkovModel(train_corpus,
                         test_corpus,
                         dic,
                         decode_type,
                         output_file):
    # build and train the model
    model = HMM(dic, decode_type)
    model.train(train_corpus)

    print("Train results:")
    pred, real = tag_corpus(model, train_corpus, output_file, dic)

    print("Tags: ", dic['id_to_tag'])
    A = compute_score(pred,real)

    # test on validation
    print("\n-----------\nTest results:")
    pred, real = tag_corpus(model, test_corpus, output_file, dic)

    print("Tags: ", dic['id_to_tag'])
    A = compute_score(pred,real)

In [50]:
# Download the dataset
import requests

def download_dataset(url, save_path):
    """
    Download dataset from URL and save it to the specified path.
    """
    response = requests.get(url)
    if response.status_code == 200:
        with open(save_path, 'wb') as f:
            f.write(response.content)
        print(f"Dataset downloaded successfully and saved at {save_path}")
    else:
        print(f"Failed to download dataset from {url}")

# Download the training dataset
train_url = "https://princeton-nlp.github.io/cos484/assignments/a2/eng.train"
train_save_path = "eng.train"
download_dataset(train_url, train_save_path)

# Download the validation dataset
val_url = "https://princeton-nlp.github.io/cos484/assignments/a2/eng.val"
val_save_path = "eng.val"
download_dataset(val_url, val_save_path)

Dataset downloaded successfully and saved at eng.train
Dataset downloaded successfully and saved at eng.val


In [51]:
train_file = 'eng.train'
test_file = 'eng.val'

# Load the training data
train_sentences = load_sentences(train_file)
train_corpus, dic = prepare_dataset(train_sentences, mode='train', word_to_id=None, tag_to_id=None)

# Load the testing data
test_sentences = load_sentences(test_file)
test_corpus = prepare_dataset(test_sentences, mode='test', word_to_id=dic['word_to_id'], tag_to_id=dic['tag_to_id'])

Found 20101 unique words (203621 in total)
Found 5 unique named entity tags


In [ ]:
runHiddenMarkovModel(
    train_corpus = train_corpus,
    test_corpus = test_corpus,
    dic = dic,
    decode_type = 'greedy',
    output_file = None
)

[0. 0. 0. 0. 0.]
[8154. 1349. 2455. 1581.  502.]


In [ ]:
runHiddenMarkovModel(
    train_corpus = train_corpus,
    test_corpus = test_corpus,
    dic = dic,
    decode_type = 'viterbi',
    output_file = None
)

In [ ]:
import os
import re
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
import collections
import codecs

In [ ]:
class MEMM(object):
    """
    Max-Entropy Markov Model (MEMM) for Named Entity Recognition.
    """

    def __init__(self, dic):
        """
        Initialize the model.
        """
        self.word_to_id = dic['word_to_id']
        self.tag_to_id = dic['tag_to_id']
        self.id_to_tag = dic['id_to_tag']
        self.tag_to_id['<start>'] = len(self.tag_to_id)
        self.id_to_tag[len(self.tag_to_id)] = '<start>'

        self.vectorizer = DictVectorizer()
        self.model = LogisticRegression()

    def prepare_data(self, corpus):
        """
        Prepare data for training the MEMM.
        """
        X = []
        y = []
        for sentence in corpus:
            for i in range(len(sentence['words'])):
                features = self.extract_features(sentence['words'], i)
                X.append(features)
                y.append(sentence['tags'][i])

        X = self.vectorizer.fit_transform(X)
        return X, y

    def extract_features(self, words, index):
        """
        Extract features for a given word in a sentence.
        """
        features = {}

        # Current word
        features['word'] = words[index]

        # Previous tag
        if index > 0:
            features['prev_tag'] = self.id_to_tag[words[index - 1]]

        return features

    def train(self, corpus):
        """
        Train the MEMM model.
        """
        X, y = self.prepare_data(corpus)
        self.model.fit(X, y)

    def decode(self, sentence):
        """
        Decode a single sentence using MEMM.
        """
        tags = []

        for i in range(len(sentence['words'])):
            features = self.extract_features(sentence['words'], i)
            X = self.vectorizer.transform([features])
            tag_id = self.model.predict(X)[0]
            tags.append(tag_id)

        return tags

In [ ]:
def runMEMM(train_corpus, test_corpus, dic, output_file):
    # Build and train the MEMM model
    model = MEMM(dic)
    model.train(train_corpus)

    # Tagging and evaluation on training data
    print("\nTrain results:")
    pred_train, real_train = tag_corpus(model, train_corpus, output_file, dic)

    # Tagging and evaluation on test data
    print("\nTest results:")
    pred_test, real_test = tag_corpus(model, test_corpus, output_file, dic)

In [ ]:
train_sentences = load_sentences(train_file)
train_corpus, dic = prepare_dataset(train_sentences, mode='train')
test_sentences = load_sentences(test_file)
test_corpus = prepare_dataset(test_sentences, mode='test', word_to_id=dic['word_to_id'], tag_to_id=dic['tag_to_id'])

In [ ]:
# Run MEMM model
runMEMM(train_corpus, test_corpus, dic, output_file=None)